In [ ]:
!pip install -q accelerate bitsandbytes tqdm sentence-transformers==2.2.2

In [ ]:
import os
import torch
import pickle
import transformers
import pandas as pd

from tqdm import tqdm
from google.colab import drive
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer

In [ ]:
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/wallpaper-defects-qa/

In [ ]:
test_file_name = 'test.csv'
test_df = pd.read_csv('data/open/' + test_file_name)
test_df.head()

In [ ]:
q_list = test_df['질문'].values.tolist()

In [ ]:
q_list[0]

# Answering

In [ ]:
model_name = "hongzoh/wdqa-v1"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    do_sample=True,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=512,
)

In [ ]:
answers = []

for q in tqdm(q_list):
    answer = pipeline(q)
    answer = answer[0].replace('\u200b', ' ')
    answers.append(answer)
    print(answer)

In [ ]:
with open(f'outputs/output_{os.path.basename(model_name)}.pickle', 'wb') as f:
    pickle.dump(answers, f)

# Embedding

In [ ]:
emb_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [ ]:
emb = emb_model.encode(answers)

In [ ]:
submission_file_name = f'sample_submission.csv'
submission_df = pd.read_csv('data/open/' + submission_file_name)
submission_df.head()

In [ ]:
assert len(emb) == len(submission_df)

In [ ]:
for test_idx in range(len(emb)):
    for vec_idx in range(512):
        submission_df.at[test_idx, 'vec_' + str(vec_idx)] = emb[test_idx][vec_idx]
submission_df.head()

In [ ]:
suffix = os.path.basename(model_name)
submission_df.to_csv(f'submission_{suffix}.csv', index=False)